In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots


from sqlalchemy import create_engine


In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'diploma'
ENGINE = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

query_users = f'''SELECT * FROM {TABLE_SCHEMA}.users'''
query_geo_daily_report = f'''SELECT * FROM {TABLE_SCHEMA}.geo_daily_report'''


# retention rate

## Считаем ret_rate user_type SQL

In [21]:
# Считаем ret_rate user_type

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

start_time = datetime.now()

users_retention_rate_user_type_sql = pd.read_sql(f''' 
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN is_organic = TRUE THEN appsflyer_id END AS organic,
            CASE WHEN is_organic != TRUE THEN appsflyer_id END AS non_organic 
        FROM {TABLE_SCHEMA}.users 
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
),
last_date AS (
-- даты всех логинов игроков
    SELECT
        DISTINCT
            appsflyer_id,
            (event_date)::date AS ld,
            organic,
            non_organic            
    FROM
        user_types
),
first_date AS (
-- дата первого входа
    SELECT
        appsflyer_id,
        organic,
        non_organic, 
        install_date::date AS fd
    FROM
        user_types
),
rate AS (
-- джоин все на все и вычитаем интервалы ld и fd
    SELECT
      -- last_date.appsflyer_id,
       last_date.organic,
       last_date.non_organic,
       ld - fd AS retention_day
    FROM
        first_date
    JOIN
        last_date ON last_date.appsflyer_id = first_date.appsflyer_id   		
)
SELECT
-- делим на кол-во игроков 0 дня для organic - 13964 и для non_organic - 6627

    retention_day,
    ROUND((COUNT(DISTINCT organic)/13964::FLOAT)::NUMERIC,2) AS ret_rate_organic,
	ROUND((COUNT(DISTINCT non_organic)/6627::FLOAT)::NUMERIC,2) as ret_rate_non_organic    
FROM 
    rate
WHERE
    1=1
    AND retention_day >= 0
GROUP BY 1
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
users_retention_rate_user_type_sql.head(3)

execution time: 0:00:01.179607


,retention_day,ret_rate_organic,ret_rate_non_organic
0,0,1.00,1.00
1,1,0.24,0.17
2,2,0.13,0.08


In [22]:
#визуализация полученных результатов

x=users_retention_rate_user_type_sql.retention_day              
y1=users_retention_rate_user_type_sql.ret_rate_organic      
y2=users_retention_rate_user_type_sql.ret_rate_non_organic  

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y2, mode='lines+markers', name='ret_rate_non_organic  '))
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines+markers', name='ret_rate_organic      '))

fig.update_layout(legend_orientation="v",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=0, r=0, t=0, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="lifetime: %{x}<br>retention: %{y}")
fig.show()

## Гипотеза  ret_rate 1 дня выше у  country_code = US

In [23]:
# Считаем кол-во игроков 0 дня для стран US-6002, MY-2669, PH-1524, UK-1575, ID-794 

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

start_time = datetime.now()

retention_rate_user_country = pd.read_sql(f''' 
WITH
    -- разделяем когорты по топ 5 странам и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN country_code = 'US' THEN appsflyer_id END AS US, --США
            CASE WHEN country_code = 'MY' THEN appsflyer_id END AS MY, --Мьянма
            CASE WHEN country_code = 'PH' THEN appsflyer_id END AS PH, --Филиппины
            CASE WHEN country_code = 'UK' THEN appsflyer_id END AS UK, --Великобритания
            CASE WHEN country_code = 'ID' THEN appsflyer_id END AS ID  --Тринидад и Тобаго
        FROM {TABLE_SCHEMA}.users 
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
),
last_date AS (
    -- даты всех логинов игроков
    SELECT
        DISTINCT
            appsflyer_id,
            (event_date)::date AS ld,
            US,
            MY,
            PH,
            UK,
            ID
    FROM
        user_types
),
first_date AS (
    -- дата первого входа
    SELECT
        appsflyer_id,
        install_date::date AS fd,
        US,
        MY,
        PH,
        UK,
        ID
    FROM
        user_types
),
rate AS (
    -- джоин все на все и вычитаем интервалы ld и fd
    SELECT
        last_date.appsflyer_id,
        last_date.ld - first_date.fd AS retention_day,
        last_date.US,
        last_date.MY,
        last_date.PH,
        last_date.UK,
        last_date.ID
    FROM
        first_date
    JOIN
        last_date ON last_date.appsflyer_id = first_date.appsflyer_id   		
)
SELECT
    -- делим на кол-во игроков 0 дня для US-6002, MY-2669, PH-1524, UK-1575, ID-794 
    retention_day,
    ROUND((COUNT(DISTINCT US)/6002::FLOAT)::NUMERIC,3) AS ret_rate_US,
	ROUND((COUNT(DISTINCT MY)/2669::FLOAT)::NUMERIC,3) as ret_rate_MY,  
    ROUND((COUNT(DISTINCT PH)/1524::FLOAT)::NUMERIC,3) AS ret_PH,
	ROUND((COUNT(DISTINCT UK)/1575::FLOAT)::NUMERIC,3) as ret_rate_UK,  
    ROUND((COUNT(DISTINCT ID)/794::FLOAT)::NUMERIC,3) as ret_rate_ID 
    
FROM 
    rate
WHERE
    1=1
    AND retention_day >= 0
GROUP BY 1
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
retention_rate_user_country.head(3)

execution time: 0:00:00.980683


,retention_day,ret_rate_us,ret_rate_my,ret_ph,ret_rate_uk,ret_rate_id
0,0,1.000,1.000,1.000,1.000,1.000
1,1,0.198,0.219,0.272,0.183,0.283
2,2,0.102,0.097,0.132,0.095,0.127


In [17]:
#визуализация полученных результатов

x=retention_rate_user_country.retention_day              
y1=retention_rate_user_country.ret_rate_us      
y2=retention_rate_user_country.ret_rate_my  
y3=retention_rate_user_country.ret_ph              
y4=retention_rate_user_country.ret_rate_uk      
y5=retention_rate_user_country.ret_rate_id  


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y1, mode='lines+markers', name='ret_rate_us  '))
fig.add_trace(go.Scatter(x=x, y=y2, mode='lines+markers', name='ret_rate_my      '))
fig.add_trace(go.Scatter(x=x, y=y3, mode='lines+markers', name='ret_ph  '))
fig.add_trace(go.Scatter(x=x, y=y4, mode='lines+markers', name='ret_rate_uk      '))
fig.add_trace(go.Scatter(x=x, y=y5, mode='lines+markers', name='ret_rate_id      '))

fig.update_layout(legend_orientation="v",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=0, r=0, t=0, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="lifetime: %{x}<br>retention: %{y}")
fig.show()

***Гипотеза  ret_rate 1 дня не подтвердилась, выше у  country_code = ID***